In [1]:
import unicodedata
import re
import json
import os
from requests import get
from typing import Dict, List, Optional, Union, cast
from bs4 import BeautifulSoup
import pandas as pd
import requests
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from time import strftime
from env import github_token, github_username
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import bts_acquire
import bts_prepare
import matplotlib.pyplot as plt
import numpy as np

/Users/brent/codeup-data-science/NLP-Github-Project/bts_acquire.py:31: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 31 of the file /Users/brent/codeup-data-science/NLP-Github-Project/bts_acquire.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(response.text)


Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories


/Users/brent/codeup-data-science/NLP-Github-Project/bts_acquire.py:43: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 43 of the file /Users/brent/codeup-data-science/NLP-Github-Project/bts_acquire.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(response.text)


Fetching links for repositories...
2 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=2
Fetching links for repositories...
3 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=3
Fetching links for repositories...
4 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=4
Fetching links for repositories...
5 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=5
Fetching links for repositories...
Get response data...
Response data obtained.
Getting repository language...
Acquired repository language.
Getting repository contents...
Repository contents acquired.
Getting readme url...
Readme url obtained.
Processing repository...
Repository processed.
Scraping github...
Scraping complete.


In [2]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository') 
        print('Fetching links for repositories...')
        git = 'https://github.com/'
        for repo in repositories:
            repository_links.append(repo.get('href')[1:])
        next_page = soup.find('a', class_='next_page').get('href')[:-1]
        ## Use this line of code to get the url for the next page.
        if page <= 4:
            url = git + next_page + str(page + 1)
            page += 1
        else:
            return repository_links


In [3]:
repository_links = get_all_repository_urls('https://github.com/orgs/apple/repositories')

Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories
Fetching links for repositories...
2 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=2
Fetching links for repositories...
3 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=3
Fetching links for repositories...
4 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=4
Fetching links for repositories...
5 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=5
Fetching links for repositories...


In [4]:
repository_links

['apple/llvm-project',
 'apple/swift-nio',
 'apple/foundationdb',
 'apple/swift-evolution',
 'apple/swift',
 'apple/swift-docc',
 'apple/swift-syntax',
 'apple/swift-package-manager',
 'apple/servicetalk',
 'apple/swift-experimental-string-processing',
 'apple/swift-tools-support-core',
 'apple/cups',
 'apple/swift-format',
 'apple/swift-docc-render',
 'apple/swift-collections',
 'apple/swift-corelibs-libdispatch',
 'apple/swift-protobuf',
 'apple/sourcekit-lsp',
 'apple/swift-argument-parser',
 'apple/swift-llbuild',
 'apple/swift-source-compat-suite',
 'apple/swift-markdown',
 'apple/swift-docc-symbolkit',
 'apple/swift-docc-render-artifact',
 'apple/swift-lmdb',
 'apple/swift-driver',
 'apple/indexstore-db',
 'apple/swift-stress-tester',
 'apple/swift-xcode-playground-support',
 'apple/swift-integration-tests',
 'apple/swift-cmark',
 'apple/swift-corelibs-foundation',
 'apple/swift-corelibs-xctest',
 'apple/swift-docker',
 'apple/swift-installer-scripts',
 'apple/swift-llbuild2',
 '

In [5]:
df = pd.read_json('data.json')

In [6]:
df = df.dropna().reset_index(drop=True)

In [7]:
df.language.value_counts()

Swift               65
Python              38
C++                 15
C                   11
JavaScript           2
HTML                 2
Jupyter Notebook     2
Java                 2
Markdown             1
R                    1
LLVM                 1
Dockerfile           1
Shell                1
Starlark             1
Name: language, dtype: int64

In [8]:
# Change 'C++' to 'C'
df.loc[df.language == 'C++', 'language'] = 'C'

In [9]:
df.language.value_counts()

Swift               65
Python              38
C                   26
JavaScript           2
HTML                 2
Jupyter Notebook     2
Java                 2
Markdown             1
R                    1
LLVM                 1
Dockerfile           1
Shell                1
Starlark             1
Name: language, dtype: int64

In [10]:
# Change values of all languages to 'Other'
df.loc[df.language.isin(['Swift','Python','C']) == False, 'language'] = 'Other'

In [11]:
df.language.value_counts()

Swift     65
Python    38
C         26
Other     14
Name: language, dtype: int64

In [12]:
# Create a list of indices where there are no readme contents
i = df[df.readme_contents == ''].index

In [13]:
# Drop observations where there is no readme content
df.drop(i, inplace=True)

In [14]:
df

,repo,language,readme_contents
0,apple/swift-argument-parser,Swift,# Swift Argument Parser\n\n## Usage\n\nBegin b...
1,apple/swift-docc,Swift,# Swift-DocC\n\nSwift-DocC is a documentation ...
2,apple/swift,C,"<img src=""https://swift.org/assets/images/swif..."
3,apple/sourcekit-lsp,Swift,# SourceKit-LSP\n\nSourceKit-LSP is an impleme...
4,apple/foundationdb,C,"<img alt=""FoundationDB logo"" src=""documentatio..."
...,...,...,...
136,apple/ml-all-pairs,Python,"<img alt=""All-Pairs example"" src=""examples/all..."
137,apple/swift-3-api-guidelines-review,Swift,# Swift 3 API Guidelines Review\n\nThis reposi...
139,apple/ccs-caldavclientlibrary,Other,README for CalDAVClientLibrary\n\nINTRODUCTION...
140,apple/ccs-pyosxframeworks,Python,Getting Started\n===============\n\nThis is a ...


In [15]:
# Use prep_article_data function to prepare the data.
df_prep = bts_prepare.prep_article_data(df, 'readme_contents')

In [16]:
df_prep[['language','lemmatized']]

,language,lemmatized
0,Swift,swift argument parser usage begin declaring ty...
1,Swift,swiftdocc swiftdocc documentation compiler swi...
2,C,img srchttpsswiftorgassetsimagesswiftsvg altsw...
3,Swift,sourcekitlsp sourcekitlsp implementation langu...
4,C,img altfoundationdb logo srcdocumentationfdb_l...
...,...,...
136,Python,img altallpairs example srcexamplesall_pairs_s...
137,Swift,swift 3 api guideline review repository part s...
139,Other,readme caldavclientlibrary introduction caldav...
140,Python,getting started python library wrap number use...


In [17]:
for readme in df_prep.lemmatized:
    print(list(pd.Series(list(nltk.ngrams(readme.split(), 2)))))

[('swift', 'argument'), ('argument', 'parser'), ('parser', 'usage'), ('usage', 'begin'), ('begin', 'declaring'), ('declaring', 'type'), ('type', 'defines'), ('defines', 'information'), ('information', 'need'), ('need', 'collect'), ('collect', 'command'), ('command', 'line'), ('line', 'decorate'), ('decorate', 'stored'), ('stored', 'property'), ('property', 'one'), ('one', 'argumentparsers'), ('argumentparsers', 'property'), ('property', 'wrapper'), ('wrapper', 'declare'), ('declare', 'conformance'), ('conformance', 'parsablecommand'), ('parsablecommand', 'add'), ('add', 'main'), ('main', 'attribute'), ('attribute', 'finally'), ('finally', 'implement'), ('implement', 'command'), ('command', 'logic'), ('logic', 'run'), ('run', 'method'), ('method', 'swift'), ('swift', 'import'), ('import', 'argumentparser'), ('argumentparser', 'main'), ('main', 'struct'), ('struct', 'repeat'), ('repeat', 'parsablecommand'), ('parsablecommand', 'flaghelp'), ('flaghelp', 'include'), ('include', 'counter'),

In [20]:
df_prep[df_prep.language == "Swift"].bigrams

0      [(swift, argument), (argument, parser), (parse...
1      [(swiftdocc, swiftdocc), (swiftdocc, documenta...
3      [(sourcekitlsp, sourcekitlsp), (sourcekitlsp, ...
5      [(img, srchttpsswiftorgassetsimagesswiftsvg), ...
7      [(swiftsyntax, swiftsyntax), (swiftsyntax, set...
                             ...                        
113    [(swiftnionghttp2support, package), (package, ...
114    [(swiftniosslsupport, package), (package, able...
115    [(swift, cluster), (cluster, membership), (mem...
118    [(swiftpmonllbuild2, repository), (repository,...
137    [(swift, 3), (3, api), (api, guideline), (guid...
Name: bigrams, Length: 65, dtype: object

In [21]:
dictionary = {'swift': df_prep[df_prep.language == "Swift"].bigrams}

In [22]:
dictionary

{'swift': 0      [(swift, argument), (argument, parser), (parse...
 1      [(swiftdocc, swiftdocc), (swiftdocc, documenta...
 3      [(sourcekitlsp, sourcekitlsp), (sourcekitlsp, ...
 5      [(img, srchttpsswiftorgassetsimagesswiftsvg), ...
 7      [(swiftsyntax, swiftsyntax), (swiftsyntax, set...
                              ...                        
 113    [(swiftnionghttp2support, package), (package, ...
 114    [(swiftniosslsupport, package), (package, able...
 115    [(swift, cluster), (cluster, membership), (mem...
 118    [(swiftpmonllbuild2, repository), (repository,...
 137    [(swift, 3), (3, api), (api, guideline), (guid...
 Name: bigrams, Length: 65, dtype: object}

In [23]:
df_prep.lemmatized[0]

'swift argument parser usage begin declaring type defines information need collect command line decorate stored property one argumentparsers property wrapper declare conformance parsablecommand add main attribute finally implement command logic run method swift import argumentparser main struct repeat parsablecommand flaghelp include counter repetition var includecounter false optionname shortandlong help number time repeat phrase var count int argumenthelp phrase repeat var phrase string mutating func run throw let repeatcount count max 1repeatcount includecounter printi phrase else printphrase argumentparser library par commandline argument instantiates command type either executes run method exit useful message argumentparser us property name type information along detail provide using property wrapper supply useful error message detailed help repeat hello count 3 hello hello hello repeat count 3 error missing expected argument phrase help phrase phrase repeat usage repeat count cou

In [24]:
pd.Series(df_prep.bigrams[0]).value_counts()

(public, api)               3
(count, count)              3
(phrase, repeat)            3
(repeat, count)             3
(hello, hello)              2
                           ..
(using, library)            1
(command, line)             1
(declaration, marked)       1
(phrase, help)              1
(option, includecounter)    1
Length: 341, dtype: int64

In [25]:
# Create a corpus of words for each category.
swift_words = ' '.join(df_prep[df_prep.language == 'Swift'].lemmatized)
c_words = ' '.join(df_prep[df_prep.language == 'C'].lemmatized)
python_words = ' '.join(df_prep[df_prep.language == 'Python'].lemmatized)
other_words = ' '.join(df_prep[df_prep.language == 'Other'].lemmatized)

In [51]:
swift_words

'swift argument parser usage begin declaring type defines information need collect command line decorate stored property one argumentparsers property wrapper declare conformance parsablecommand add main attribute finally implement command logic run method swift import argumentparser main struct repeat parsablecommand flaghelp include counter repetition var includecounter false optionname shortandlong help number time repeat phrase var count int argumenthelp phrase repeat var phrase string mutating func run throw let repeatcount count max 1repeatcount includecounter printi phrase else printphrase argumentparser library par commandline argument instantiates command type either executes run method exit useful message argumentparser us property name type information along detail provide using property wrapper supply useful error message detailed help repeat hello count 3 hello hello hello repeat count 3 error missing expected argument phrase help phrase phrase repeat usage repeat count cou

In [26]:
swift_bigrams = pd.Series(list(nltk.ngrams(swift_words.split(), 2)))
c_bigrams = pd.Series(list(nltk.ngrams(c_words.split(), 2)))
python_bigrams = pd.Series(list(nltk.ngrams(python_words.split(), 2)))
other_bigrams = pd.Series(list(nltk.ngrams(other_words.split(), 2)))

In [53]:
swift_bigrams

0          (swift, argument)
1         (argument, parser)
2            (parser, usage)
3             (usage, begin)
4         (begin, declaring)
                ...         
29885        (version, used)
29886       (used, generate)
29887    (generate, current)
29888        (current, data)
29889             (data, 72)
Length: 29890, dtype: object

In [27]:
df_prep['bigrams'] = [list(pd.Series(list(nltk.ngrams(readme.split(),2)))) for readme in df_prep.lemmatized]

In [ ]:
# Use bigram code


In [50]:
df_prep

,repo,language,readme_contents,clean,stemmed,lemmatized,bigrams
0,apple/swift-argument-parser,Swift,# Swift Argument Parser\n\n## Usage\n\nBegin b...,swift argument parser usage begin declaring ty...,swift argument parser usag begin declar type d...,swift argument parser usage begin declaring ty...,"[(swift, argument), (argument, parser), (parse..."
1,apple/swift-docc,Swift,# Swift-DocC\n\nSwift-DocC is a documentation ...,swiftdocc swiftdocc documentation compiler swi...,swiftdocc swiftdocc document compil swift fram...,swiftdocc swiftdocc documentation compiler swi...,"[(swiftdocc, swiftdocc), (swiftdocc, documenta..."
2,apple/swift,C,"<img src=""https://swift.org/assets/images/swif...",img srchttpsswiftorgassetsimagesswiftsvg altsw...,img srchttpsswiftorgassetsimagesswiftsvg altsw...,img srchttpsswiftorgassetsimagesswiftsvg altsw...,"[(img, srchttpsswiftorgassetsimagesswiftsvg), ..."
3,apple/sourcekit-lsp,Swift,# SourceKit-LSP\n\nSourceKit-LSP is an impleme...,sourcekitlsp sourcekitlsp implementation langu...,sourcekitlsp sourcekitlsp implement languag se...,sourcekitlsp sourcekitlsp implementation langu...,"[(sourcekitlsp, sourcekitlsp), (sourcekitlsp, ..."
4,apple/foundationdb,C,"<img alt=""FoundationDB logo"" src=""documentatio...",img altfoundationdb logo srcdocumentationfdb_l...,img altfoundationdb logo srcdocumentationfdb_l...,img altfoundationdb logo srcdocumentationfdb_l...,"[(img, altfoundationdb), (altfoundationdb, log..."
...,...,...,...,...,...,...,...
136,apple/ml-all-pairs,Python,"<img alt=""All-Pairs example"" src=""examples/all...",img altallpairs example srcexamplesall_pairs_s...,img altallpair exampl srcexamplesall_pairs_sur...,img altallpairs example srcexamplesall_pairs_s...,"[(img, altallpairs), (altallpairs, example), (..."
137,apple/swift-3-api-guidelines-review,Swift,# Swift 3 API Guidelines Review\n\nThis reposi...,swift 3 api guidelines review repository part ...,swift 3 api guidelin review thi repositori par...,swift 3 api guideline review repository part s...,"[(swift, 3), (3, api), (api, guideline), (guid..."
139,apple/ccs-caldavclientlibrary,Other,README for CalDAVClientLibrary\n\nINTRODUCTION...,readme caldavclientlibrary introduction caldav...,readm caldavclientlibrari introduct caldavclie...,readme caldavclientlibrary introduction caldav...,"[(readme, caldavclientlibrary), (caldavclientl..."
140,apple/ccs-pyosxframeworks,Python,Getting Started\n===============\n\nThis is a ...,getting started python library wraps number us...,get start thi python librari wrap number use o...,getting started python library wrap number use...,"[(getting, started), (started, python), (pytho..."


In [28]:
swift_bigrams.value_counts()

(swift, package)                   64
(standard, library)                36
(package, manager)                 35
(distributed, actor)               33
(swift, project)                   28
                                   ..
(including, integer)                1
(processor, cmpxchg16b)             1
(either, objectivec)                1
(superrecordduration, security)     1
(se0270_rangeset, package)          1
Length: 22495, dtype: int64

In [29]:
df_prep[df_prep.language == "Swift"].bigrams

0      [(swift, argument), (argument, parser), (parse...
1      [(swiftdocc, swiftdocc), (swiftdocc, documenta...
3      [(sourcekitlsp, sourcekitlsp), (sourcekitlsp, ...
5      [(img, srchttpsswiftorgassetsimagesswiftsvg), ...
7      [(swiftsyntax, swiftsyntax), (swiftsyntax, set...
                             ...                        
113    [(swiftnionghttp2support, package), (package, ...
114    [(swiftniosslsupport, package), (package, able...
115    [(swift, cluster), (cluster, membership), (mem...
118    [(swiftpmonllbuild2, repository), (repository,...
137    [(swift, 3), (3, api), (api, guideline), (guid...
Name: bigrams, Length: 65, dtype: object

In [30]:
# Get length of readme files.
len(swift_words), len(c_words), len(python_words), len(other_words)

(278029, 75046, 151836, 34884)

In [31]:
# Top 20 Swift words
pd.Series(swift_words.split()).value_counts().head(20)

swift             772
package           300
use               216
project           213
library           205
using             183
build             172
dependency        150
type              143
test              143
file              142
run               140
implementation    138
version           133
swiftnio          132
api               130
code              129
work              128
let               119
xcode             119
dtype: int64

In [32]:
# Top 20 C words
pd.Series(c_words.split()).value_counts().head(20)

build         157
swift         119
file           95
kernel         73
use            70
make           66
code           65
support        60
crypto         47
package        46
project        46
repository     46
using          45
install        42
library        39
available      39
llbuild        39
system         38
header         37
1              36
dtype: int64

In [33]:
# Top 20 Python words
pd.Series(python_words.split()).value_counts().head(20)

&#9;         1574
python        158
file          129
dataset       123
data          122
model         118
element       108
test          107
use            94
training       91
used           87
scene          86
run            81
value          78
using          77
following      76
code           74
example        70
parameter      70
project        68
dtype: int64

In [34]:
# Top 20 Other words
pd.Series(other_words.split()).value_counts().head(20)

password         52
swift            36
website          28
project          27
tensorflow       25
user             23
use              23
run              23
data             22
file             21
code             20
information      20
rule             19
documentation    18
manager          18
using            17
model            16
ml               16
see              16
following        16
dtype: int64

In [35]:
# Top 20 Swift bigrams
swift_bigrams.value_counts().head(20)

(swift, package)              64
(standard, library)           36
(package, manager)            35
(distributed, actor)          33
(swift, project)              28
(backend, implementation)     27
(swift, import)               26
(logging, backend)            26
(please, see)                 23
(public, api)                 23
(header, field)               23
(command, line)               23
(icloud, container)           22
(make, sure)                  22
(metric, backend)             22
(swift, build)                21
(source, code)                20
(log, message)                19
(protocol, implementation)    18
(derivative, work)            18
dtype: int64

In [36]:
# Top 20 C bigrams
c_bigrams.value_counts().head(20)

(swift, crypto)        45
(header, file)         26
(build, system)        18
(turi, create)         12
(x86_64, build)        10
(command, line)        10
(open, source)          9
(user, level)           9
(1, install)            8
(channel, binding)      8
(defined, enclosed)     7
(would, like)           7
(cmake, g)              7
(getting, started)      7
(xnu, kernel)           7
(lpadmin, p)            6
(installs, header)      6
(kernel, extension)     6
(rpm, package)          6
(file, available)       6
dtype: int64

In [37]:
# Top 20 Python bigrams
python_bigrams.value_counts().head(20)

(&#9;, &#9;)            933
(&#9;, element)          71
(arg, &#9;)              56
(&#9;, arg)              54
(&#9;, argument)         35
(&#9;, set)              31
(&#9;, verify)           30
(data, parameter)        28
(verify, &#9;)           28
(bounding, box)          26
(camera, trajectory)     23
(feature, &#9;)          22
(test, &#9;)             19
(following, command)     18
(pip, install)           18
(&#9;, attribute)        18
(example, &#9;)          17
(hypersim, dataset)      15
(command, python)        15
(&#9;, example)          15
dtype: int64

In [38]:
# Top 20 Other bigrams
other_bigrams.value_counts().head(20)

(password, manager)         17
(ml, compute)               15
(pull, request)              8
(change, password)           6
(password, rule)             6
(project, maintainer)        6
(covid19, spread)            5
(eager, mode)                5
(source, code)               5
(getting, started)           5
(cell, phone)                5
(mobility, covid19)          5
(feature, request)           5
(tensorflow, addons)         5
(log, file)                  4
(documentation, archive)     4
(bug, report)                4
(tensorflow, tensorflow)     4
(npm, run)                   4
(jupyter, notebook)          4
dtype: int64

In [39]:
# A simple way to calculate idf for demonstration. Note that this
# function relies on the globally defined documents variable.
def idf(word):
    n_occurences = sum([1 for doc in dictionary.values() if word in doc])
    return len(dictionary) / n_occurences

# Get a list of the unique words
unique_words = pd.Series(' '.join(dictionary.values()).split()).unique()

# put the unique words into a data frame
(pd.DataFrame(dict(word=unique_words))
 # calculate the idf for each word
 .assign(idf=lambda df: df.word.apply(idf))
 # sort the data for presentation purposes
 .set_index('word')
 .sort_values(by='idf', ascending=False))

TypeError: sequence item 0: expected str instance, Series found

In [40]:
tfs = []

# We'll caclulate the tf-idf value for every word across every document

# Start by iterating over all the documents
for lang, words in dictionary.items():
    # We'll make a data frame that contains the tf for every word in every document
    df = (pd.Series(words.split())
          .value_counts()
          .reset_index()
          .set_axis(['word', 'raw_count'], axis=1, inplace=False)
          .assign(tf=lambda df: df.raw_count / df.shape[0])
          .drop(columns='raw_count')
          .assign(lang=lang))
    # Then add that data frame to our list
    tfs.append(df)

# We'll then concatenate all the tf values together.
#df = (pd.concat(tfs)
# # calculate the idf value for each word
# .assign(idf=lambda df: df.word.apply(idf))
# # then use the if and idf values to calculate tf-idf 
# .assign(tf_idf=lambda df: df.idf * df.tf)
# .drop(columns=['tf', 'idf'])
# .sort_values(by='tf_idf', ascending=False))

# We'll then concatenate all the tf values together.
df =(pd.concat(tfs)
 # calculate the idf value for each word
 .assign(idf=lambda df: df.word.apply(idf))
 # then use the if and idf values to calculate tf-idf 
.assign(tf_idf=lambda df: df.idf * df.tf)
.drop(columns=['tf', 'idf'])
.sort_values(by='tf_idf', ascending=False)
.pipe(lambda df: pd.crosstab(df.lang, df.word, values=df.tf_idf, aggfunc=lambda x: x))
.fillna(0))

AttributeError: 'Series' object has no attribute 'split'

In [41]:
len(df.columns)

6

## Try TF-IDF with scikit-learn

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df.columns)
y = pd.Series(df.index)

In [43]:
X

<6x6 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [44]:
y

0        0
1        1
2        2
3        3
4        4
      ... 
134    136
135    137
136    139
137    140
138    141
Length: 139, dtype: int64

In [45]:
df = pd.DataFrame(tfidfs.todense(), columns=tfidf.get_feature_names())

NameError: name 'tfidfs' is not defined

In [46]:
df.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            131, 132, 133, 134, 135, 136, 137, 139, 140, 141],
           dtype='int64', length=139)

In [47]:
X, y

(<6x6 sparse matrix of type '<class 'numpy.float64'>'
 	with 6 stored elements in Compressed Sparse Row format>,
 0        0
 1        1
 2        2
 3        3
 4        4
       ... 
 134    136
 135    137
 136    139
 137    140
 138    141
 Length: 139, dtype: int64)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

lm = LogisticRegression().fit(X_train, y_train)

train['predicted'] = lm.predict(X_train)
test['predicted'] = lm.predict(X_test)

ValueError: Found input variables with inconsistent numbers of samples: [6, 139]

In [49]:
train

NameError: name 'train' is not defined

In [ ]:
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.predicted, train.actual))
print('---')
print(classification_report(train.actual, train.predicted))

In [ ]:
print('Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(test.predicted, test.actual))
print('---')
print(classification_report(test.actual, test.predicted))